In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd

In [2]:
import sys
sys.path.append("..")
from simdb.databaseModel import *
import simdb.databaseAPI as api

In [3]:
# db_path = 'test_WORKS.db'
db_path = '../examples/setup_databases/andrej/andrej_raw.db'

In [4]:
from sqlalchemy import or_, and_, or_

## For migration to databaseAPI:

In [ ]:
session = api.connect_database(db_path)

In [ ]:
gk = GroupKeywords(name="Chain1", value="resid 1-72")

In [ ]:
session.query(Groups).filter(Groups.name == "Group 1").one()

In [ ]:
session.query(Main).join(association_main_groups).join(Groups).filter(Groups.name == "Group 2").count()

In [ ]:
api.get_all_groups(session=session)

In [ ]:
query = session.query(Main.entry_id).join(association_main_groups).join(Groups).join(Keywords)

In [ ]:
query.filter(Groups.name == "Group 2", Keywords.name == "linker").count()

In [ ]:
query.filter(Groups.name == "Group 1", Keywords.name == "seq").count()

## HowTo

### Load Entry Table

In [ ]:
with api.session_handler(db_path) as s:
    table = api.get_entry_table(s) # load whole table
table

In [ ]:
# f = and_(Main.keywords.any(name="linker", value="k48"), Main.keywords.any(name="ff", value="MARTINI"))
f = Main.keywords.any(name="expansion_type", value="cg_min_rnd")
with api.session_handler(db_path) as s:
    table = api.get_entry_table(s, apply_filter=f) # load whole table
table

In [ ]:
f = Main.keywords.any(name="linker", value="k48")
with api.session_handler(db_path) as s:
    table = api.get_entry_table(s, apply_filter=f) # load whole table
table

In [ ]:
f = Main.keywords.any(name="temp", value="300")
with api.session_handler(db_path) as s:
    table = api.get_entry_table(s, apply_filter=f) # load whole table
table

### Obtain data for certain entry

In [ ]:
# opbtain all details as dictionary
entry_id = "2017_04_21_fat10_FL_exp_03"

with api.session_handler(db_path) as s:
    details = api.get_entry_details(s, entry_id)
details

### Select entries

In [ ]:
table[selectByTag(table, "test")] # select by tag

In [ ]:
table[selectByKeyword(table, "ff", "MARTINI")] # select by keyword

In [ ]:
# combinations of selections
ff       = selectByKeyword(table, "ff", "MARTINI")
chainlen = selectByKeyword(table, "chainlen", "1")
tags     = selectByTag(table, "test2")

table[ff & chainlen & tags]

In [ ]:
# another example
ff       = selectByKeyword(table, "ff", "MARTINI")
chainlen = selectByKeyword(table, "chainlen", "2")
linker1  = selectByKeyword(table, "linker", "k48")
linker2  = selectByKeyword(table, "linker", "k63")
number   = selectByKeyword(table, "number", "01")

table[ff & chainlen & (linker1 | linker2) & ~number]

### Working with groups

In [9]:
with api.session_handler(db_path) as s:
    table = api.get_entry_table(s, keyword_names=["linker"])
    
    for entry_id in table.entry_id:
        api.add_to_group(s, entry_id, "Group 1")
    
    s.commit()

In [ ]:
f = Main.keywords.any(name="linker", value="k48")
with api.session_handler(db_path) as s:
    table = api.get_entry_table(s, apply_filter=f) # load whole table
table

In [ ]:
[api.add_group(db_path, entry_id, "Group 2") for entry_id in table.entry_id]

In [ ]:
# add some entries to a group
sel = table[selectByKeyword(table, "linker", "k48")]
[add_group(db_path, entry_id, "Group 1") for entry_id in sel.entry_id]

# a second group
sel = table[selectByKeyword(table, "ff", "MARTINI")]
[add_group(db_path, entry_id, "Group 2") for entry_id in sel.entry_id]

In [ ]:
# get entries of group 1
entry_group_table1 = get_entry_table(db_path, group_names=["Group 1"], columns=["entry_id", "path"])
print("{} entries loaded.".format(entry_group_table1.shape[0]))

# get entries of group 2
entry_group_table2 = get_entry_table(db_path, group_names=["Group 2"], columns=["entry_id", "path"])
print("{} entries loaded.".format(entry_group_table2.shape[0]))

In [ ]:
# get entries from both groups
# some entries might be in both groups
merged_table = get_entry_table(db_path, group_names=["Group 1", "Group 2"], columns=["entry_id", "path"])
print("Merge results in {} entries.".format(merged_table.shape[0]))

In [ ]:
# remove some etnries from Group 1
sel = table[selectByKeyword(table, "ff", "MARTINI")]
[remove_group(db_path, entry_id, "Group 1") for entry_id in sel.entry_id]

In [ ]:
session = api.connect_database(db_path=db_path)
# groups = api.get_all_groups(session=session)

### Usefull for backend / viewer

In [ ]:
# open database (in SQLAlchemy we open a session)
# s = openDatabase(db_path) 
# one should not keep the DB open

# better:
with api.session_handler(db_path) as s:
    keywords = api.get_all_keywords(s)  # get all keyword with the values they have
pd.Series(keywords)

In [ ]:
with session_handler(db_path) as s:
    groups = get_all_groups(s)  # get all groups in DB
groups

## Add, edit remove Metadata

In [ ]:
s = open_database(db_path)

In [ ]:
entry = s.query(Main).filter(Main.entry_id == entry_id).first()
# entry.keywords

In [ ]:
entry.meta.all()

In [ ]:
mg = entry.meta.filter_by(name="barostat").first()

In [ ]:
mg.entries.all()

In [ ]:
entry.groups

In [ ]:
s.close()

### Tags

In [ ]:
add_tag(db_path, entry_id, "my_tag")

In [ ]:
remove_tag(db_path, entry_id, "my_tag")

### Keywords

In [ ]:
# add two keywords to certain entry
add_keyword(db_path, entry_id, keyword1="bla", keyword2="blub")

In [ ]:
# alter keyword1 of entry
alter_keyword(db_path, entry_id, keyword1="my_value")

In [ ]:
# remove keyword2 if it is "blub" and force remove keyword1
remove_keyword(db_path, entry_id, keyword1=None, keyword2="blub")

### Meta information

In [ ]:
meta_group_name = "More Information"

In [ ]:
# add some information to entry
add_meta_group(db_path, entry_id, meta_group_name)
add_meta_data(db_path, entry_id, meta_group_name, 
              run="the best one", 
              importance="rather large", 
              version="quite new")

In [ ]:
alter_meta_data(db_path, entry_id, meta_group_name,
               run = "actually not that good")

In [ ]:
# remove some meta data entries from group
remove_meta_data(db_path, entry_id, meta_group_name,
                run=None,
                version="quite new")

# or remove the whole group
remove_meta_group(db_path, entry_id, meta_group_name)

In [ ]:
# check out changes
pd.DataFrame(get_entry_details(db_path, entry_id)['meta'])

### Running functions

## Deva

* Add tags/keywords to all simulations in pandas table
* Add simulations to DB from pandas table